## Exploratory Data Analysis ##
Although the Kaggle competition uses Auc to measure success in this project we decided to concentrate on Sensitivity as we want to identify as much of the West Nile virus as possible.  We will not be able to judge our sensitivity score on the test data as it will evaluate us on Auc but we will still treat the test data as though we are evaluating it by Sensitivity

In [1]:
# Installed geopy, which is a tool to help determine distances between objects.  We used this below to determine
# the distance between if West Nile Virus is presenet to those cases it wasn't, by year
!pip install geopy

In [2]:
# Import libraries
import pandas as pd
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import time

In [3]:
# Import csv data files and assign to variable
spray = pd.read_csv('./data/spray.csv')
weather = pd.read_csv('./data/weather.csv')
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
# Use the imported column names and lowercase all letters and add an underscore where spaces exist between words
train.columns = [col.lower().replace(' ', '_') for col in train.columns]
test.columns = [col.lower().replace(' ', '_') for col in test.columns]
spray.columns = [col.lower().replace(' ', '_') for col in spray.columns]
weather.columns = [col.lower().replace(' ', '_') for col in weather.columns]

In [5]:
#create baseline file to upload to Kaggle
baseline=test.loc[:,("id")]
baseline['wnvpresent'] = 0
baseline.to_csv('./Data/baselinesubmission.csv', index=False)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### Baseline Score

We created a baseline file, where there were no West Nile Virus predictions made (all zero's), to upload to Kaggle to get an idea what the score would be, according to their measurements.  While it turned out that our score was .50, that wasn't much of a surprise since a mosquito either has West Nile or does not.

In [6]:
#baseline on train
1-train['wnvpresent'].mean()

0.9475537787930707

In [7]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
date         14835 non-null object
time         14251 non-null object
latitude     14835 non-null float64
longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [8]:
#looked for null values in spray dataset
spray[spray.isnull().any(axis=1)]

,date,time,latitude,longitude
1030,2011-09-07,NaN,41.987092,-87.794286
1031,2011-09-07,NaN,41.987620,-87.794382
1032,2011-09-07,NaN,41.988004,-87.794574
1033,2011-09-07,NaN,41.988292,-87.795486
1034,2011-09-07,NaN,41.988100,-87.796014
1035,2011-09-07,NaN,41.986372,-87.794862
1036,2011-09-07,NaN,41.986228,-87.795582
1037,2011-09-07,NaN,41.984836,-87.793998
1038,2011-09-07,NaN,41.984836,-87.794670
1039,2011-09-07,NaN,41.984884,-87.795198


In [9]:
# Dropped time column from spray because it isn't contained in any other datasets, and has a high number of missing
# values
spray.drop(columns='time', inplace=True)

In [10]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [11]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
station        2944 non-null int64
date           2944 non-null object
tmax           2944 non-null int64
tmin           2944 non-null int64
tavg           2944 non-null object
depart         2944 non-null object
dewpoint       2944 non-null int64
wetbulb        2944 non-null object
heat           2944 non-null object
cool           2944 non-null object
sunrise        2944 non-null object
sunset         2944 non-null object
codesum        2944 non-null object
depth          2944 non-null object
water1         2944 non-null object
snowfall       2944 non-null object
preciptotal    2944 non-null object
stnpressure    2944 non-null object
sealevel       2944 non-null object
resultspeed    2944 non-null float64
resultdir      2944 non-null int64
avgspeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [12]:
weather['tavg'].value_counts()

73    138
77    117
70    117
75    110
71    109
74    107
72    104
69    103
78    102
76    100
68     99
79     98
66     93
67     89
61     88
64     86
65     84
80     84
63     81
57     67
62     66
60     61
50     57
81     55
58     49
53     49
54     48
55     48
82     48
56     46
52     46
59     45
51     36
83     34
49     29
45     28
46     24
47     24
84     21
44     19
48     17
86     16
85     16
42     15
43     12
M      11
87      9
41      7
40      5
39      4
91      4
88      4
89      4
38      2
90      2
36      2
37      2
93      1
94      1
92      1
Name: tavg, dtype: int64

In [13]:
# Dropped all t-averages with missing value 'M' because there were additional missing values in each of these rows.
# Only 11 were missing.
weather = (weather[weather['tavg'] != 'M'])

In [14]:
# Look at Missing values is the depart column of weather
weather[weather['depart'] == 'M']

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,,M,M,M,T,29.46,30.09,11.2,7,11.5
11,2,2007-05-06,68,52,60,M,30,46,5,0,...,,M,M,M,0.00,29.62,30.28,13.8,10,14.5
13,2,2007-05-07,84,50,67,M,39,53,0,2,...,,M,M,M,0.00,29.44,30.12,8.5,17,9.9
15,2,2007-05-08,80,60,70,M,57,63,0,5,...,HZ,M,M,M,T,29.36,30.02,2.5,8,5.4
17,2,2007-05-09,76,63,70,M,60,63,0,5,...,BR HZ,M,M,M,0.02,29.28,29.93,3.9,7,5.9
19,2,2007-05-10,83,59,71,M,52,61,0,6,...,BR HZ,M,M,M,0.00,29.26,29.91,2.0,9,3.9
21,2,2007-05-11,73,49,61,M,44,51,4,0,...,,M,M,M,0.00,29.39,30.03,11.7,36,12.8


In [15]:
# There are 0.498 missing values from weather depart column
weather['depart'].value_counts(normalize=True)

M      0.498125
 2     0.031708
-1     0.028640
-2     0.027276
 5     0.026253
 1     0.025912
 7     0.025912
 3     0.025571
 0     0.025230
-3     0.024548
 4     0.024207
 6     0.022844
 8     0.020116
-5     0.019434
-4     0.019093
-6     0.017047
 9     0.016025
10     0.015684
-8     0.014661
-7     0.010228
12     0.009547
11     0.009547
-9     0.008524
13     0.007842
14     0.007501
-10    0.007501
15     0.005114
16     0.004091
-11    0.003409
-12    0.002728
17     0.002387
-14    0.002046
18     0.002046
-13    0.001705
19     0.001364
20     0.001364
-16    0.001023
-15    0.001023
22     0.001023
21     0.000682
-17    0.000682
23     0.000341
Name: depart, dtype: float64

In [16]:
# Check for null values in Weather
weather[weather.isnull().any(axis=1)]

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed


In [17]:
weather.shape

(2933, 22)

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
date                      10506 non-null object
address                   10506 non-null object
species                   10506 non-null object
block                     10506 non-null int64
street                    10506 non-null object
trap                      10506 non-null object
addressnumberandstreet    10506 non-null object
latitude                  10506 non-null float64
longitude                 10506 non-null float64
addressaccuracy           10506 non-null int64
nummosquitos              10506 non-null int64
wnvpresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [19]:
# Check for null values in train data
train[train.isnull().any(axis=1)]

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent


In [20]:
train.columns

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'nummosquitos', 'wnvpresent'],
      dtype='object')

#### Leakage

The train dataset contains some duplicate trap records as traps with mosquito counts over 50 are entered in multiple rows.  We are dropping the duplicate rows below.  We do not drop from the test data because we need to predict every observation.

In [21]:
train.drop_duplicates(subset=['trap','date','species'], inplace=True)


In [22]:
# Convert dates to datetime
spray['date']   = pd.to_datetime(spray['date'])
weather['date'] = pd.to_datetime(weather['date'])
train['date']   = pd.to_datetime(train['date'])
test['date']    = pd.to_datetime(test['date'])

In [23]:
test['date'].head()

0   2008-06-11
1   2008-06-11
2   2008-06-11
3   2008-06-11
4   2008-06-11
Name: date, dtype: datetime64[ns]

In [24]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [25]:
# Split dates into individual categories test file
test['year'] = [str(date)[0:4] for date in test.date]
test['month'] = [str(date)[5:7] for date in test.date]
test['day'] = [str(date)[8:10] for date in test.date]

In [26]:
# Split dates into individual categories train file
train['year'] = [str(date)[0:4] for date in train.date]
train['month'] = [str(date)[5:7] for date in train.date]
train['day'] = [str(date)[8:10] for date in train.date]

In [27]:
# Combine weather stations 1 and 2 based on date
weather1 = weather[weather['station'] == 1]
weather2 = weather[weather['station'] == 2]
weather1.drop(columns='station', inplace=True)
weather2.drop(columns='station', inplace=True)
weather = weather1.merge(weather2, on='date')

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [28]:
# Merge train and test datasets with weather dataset based on the date field
train=train.merge(weather, on='date')
test=test.merge(weather, on='date')

In [29]:
# Get rid of alpha characters in traps to create number field
train['trap']=[trap.replace("T","") for trap in train['trap']]
test['trap']=[trap.replace("T","") for trap in test['trap']]
train['trap']=[trap.replace("A","") for trap in train['trap']]
test['trap']=[trap.replace("A","") for trap in test['trap']]
train['trap']=[trap.replace("B","") for trap in train['trap']]
test['trap']=[trap.replace("B","") for trap in test['trap']]
train['trap']=[trap.replace("C","") for trap in train['trap']]
test['trap']=[trap.replace("C","") for trap in test['trap']]

In [30]:
# Change field type for trap from object to int in test and train
train['trap']=pd.to_numeric(train['trap'])
test['trap']=pd.to_numeric(test['trap'])

In [31]:
# Make dummy columns for species type in test and train
train=pd.get_dummies(train, columns=['species'], drop_first=True)
test=pd.get_dummies(test, columns=['species'], drop_first=True)

In [32]:
train.dtypes

date                              datetime64[ns]
address                                   object
block                                      int64
street                                    object
trap                                       int64
addressnumberandstreet                    object
latitude                                 float64
longitude                                float64
addressaccuracy                            int64
nummosquitos                               int64
wnvpresent                                 int64
year                                      object
month                                     object
day                                       object
tmax_x                                     int64
tmin_x                                     int64
tavg_x                                    object
depart_x                                  object
dewpoint_x                                 int64
wetbulb_x                                 object
heat_x              

In [33]:
# Create lowercase characters and underscore species dummy columns
train.columns = [col.lower().replace(' ', '_') for col in train.columns]
test.columns = [col.lower().replace(' ', '_') for col in test.columns]

In [34]:
train['trap'].head()

0     2
1     2
2     7
3    15
4    15
Name: trap, dtype: int64

In [35]:
train['addressnumberandstreet'].head()

0    4100  N OAK PARK AVE, Chicago, IL
1    4100  N OAK PARK AVE, Chicago, IL
2     6200  N MANDELL AVE, Chicago, IL
3      7900  W FOSTER AVE, Chicago, IL
4      7900  W FOSTER AVE, Chicago, IL
Name: addressnumberandstreet, dtype: object

In [36]:
train['address'].head()

0    4100 North Oak Park Avenue, Chicago, IL 60634,...
1    4100 North Oak Park Avenue, Chicago, IL 60634,...
2    6200 North Mandell Avenue, Chicago, IL 60646, USA
3      7900 West Foster Avenue, Chicago, IL 60656, USA
4      7900 West Foster Avenue, Chicago, IL 60656, USA
Name: address, dtype: object

In [37]:
train['wnvpresent'].head()

0    0
1    0
2    0
3    0
4    0
Name: wnvpresent, dtype: int64

In [38]:
train['trap'].value_counts()

900    154
135    147
2      147
151    143
115    143
48     139
31     139
54     136
90     135
138    129
209    128
114    126
11     125
94     123
73     123
158    121
212    120
128    119
28     113
3      113
8      113
47     110
102    109
66     108
30     108
74     107
46     106
159    104
63     103
61     101
      ... 
150     22
19      22
206     21
219     21
141     21
156     20
154     20
162     20
75      18
70      17
142     17
71      17
50      16
34      16
51      15
97      15
157     14
1       14
44      14
60      14
149     14
5       14
4       12
72      11
229      8
238      7
78       6
76       5
237      3
40       2
Name: trap, Length: 134, dtype: int64

In [39]:
test['trap'].value_counts()

90     3051
218    3051
2      2377
200    2291
54     1552
128    1534
65     1532
9      1528
94     1521
35     1520
900    1468
8       822
11      814
27      803
151     797
903     784
28      783
231     781
12      781
3       780
220     776
115     776
73      776
16      775
135     774
223     774
63      774
158     773
212     773
102     772
       ... 
25      760
107     760
88      760
215     760
44      760
150     760
86      760
149     760
85      760
43      760
91      760
84      760
155     760
219     760
19      760
146     760
234     760
18      760
92      760
156     760
81      760
17      760
157     760
232     760
40      760
143     760
79      760
15      760
206     760
154     760
Name: trap, Length: 135, dtype: int64

In [40]:
# Change types of date fields in test and train
train['year'] = train['year'].astype(float)
test['year'] = test['year'].astype(float)
train['month'] = train['month'].astype(float)
test['month'] = test['month'].astype(float)
train['day'] = train['day'].astype(float)
test['day'] = test['day'].astype(float)

In [41]:
list = [12,24, 36, 47]
sum(list[0:3])/len(list)

18.0

### Geopy

Using Geopy, below we are using the latitude and longitude, based on the corresponding year, and creating a function that calculates the distance between positive West Nile Virus samples found in mosquitoes vs. the mosquitoes that did not test positive for the West Nile.  We repeated the process for each year in the train and test data, that will be used as features later on in our model.  The comments are the same for each function below

In [42]:
%%time

# Using 2007 Train data, find the distance between mosquitos that tested positive for WNV vs. those that did not

# Filter by year
distance_2007 = train[train['year'] == 2007.0]
features = ['latitude', 'longitude']

distance_2007 = distance_2007[features]
distance_2007 = distance_2007.apply(tuple, axis = 1)
train['wnvpresent'] = train['wnvpresent'].astype(float)

# Find if WNV present for year
westnile_2007 = (train[train['wnvpresent'] == 1]) 
westnile_2007 = westnile_2007[westnile_2007['year'] == 2007.0]
westnile_2007 = westnile_2007[features]
westnile_2007 = westnile_2007.apply(tuple, axis = 1)

# Iterate through each distance, and if it has WNV present, append it to list

results_2007   = []
results_2007_2 = []
results_2007_3 = []

for i in distance_2007:
    dists_2007 = []

    for j in westnile_2007:
        
       # Use Geopy to find the difference in distances, in miles 
        dist_2007 = (geodesic(i, j).miles)
        if dist_2007 > 0:
            dists_2007.append(dist_2007)
   # sort and add minimum and mean minumum distances
    dists_2007.sort()
    results_2007.append(min(dists_2007))
    results_2007_2.append(sum(dists_2007[0:3])/3)
    results_2007_3.append(sum(dists_2007[0:5])/5)
    
# Create dataframe    
train_2007=pd.DataFrame(results_2007, columns = ['distance'])
train_2007['distance_3']  = results_2007_2
train_2007['distance_5']  = results_2007_3                                              
train_07 = train[train['year'] == 2007]
train_07.reset_index(inplace = True)
train_7 = pd.concat([train_07, train_2007], axis = 1)

CPU times: user 1min 4s, sys: 37.6 ms, total: 1min 4s
Wall time: 1min 4s


In [43]:
train_7.head()

,index,date,address,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,...,avgspeed_y,species_culex_pipiens,species_culex_pipiens/restuans,species_culex_restuans,species_culex_salinarius,species_culex_tarsalis,species_culex_territans,distance,distance_3,distance_5
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,7.4,0,1,0,0,0,0,1.050839,1.117797,1.176855
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,7.4,0,0,1,0,0,0,1.050839,1.117797,1.176855
2,2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,7,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,7.4,0,0,1,0,0,0,1.021493,1.021493,1.050681
3,3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,7.4,0,1,0,0,0,0,0.822410,0.822410,1.218078
4,4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,7.4,0,0,1,0,0,0,0.822410,0.822410,1.218078


In [44]:
%%time

# Using 2009 Train data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2009 = train[train['year'] == 2009.0]
features = ['latitude', 'longitude']

distance_2009 = distance_2009[features]
distance_2009 = distance_2009.apply(tuple, axis = 1)
train['wnvpresent'] = train['wnvpresent'].astype(float)

westnile_2009 = (train[train['wnvpresent'] == 1]) 
westnile_2009 = westnile_2009[westnile_2009['year'] == 2009.0]
westnile_2009 = westnile_2009[features]
westnile_2009 = westnile_2009.apply(tuple, axis = 1)
westnile_2009.head()

results_2009   = []
results_2009_2 = []
results_2009_3 = []

for i in distance_2009:
    dists_2009 = []

    for j in westnile_2009:

        dist_2009 = (geodesic(i, j).miles)
        if dist_2009 > 0:
            dists_2009.append(dist_2009)
    
    results_2009.append(min(dists_2009))
    results_2009_2.append(sum(dists_2009[0:3])/3)
    results_2009_3.append(sum(dists_2009[0:5])/5)
    
# Create dataframe    
train_2009=pd.DataFrame(results_2009, columns = ['distance'])
train_2009['distance_3']  = results_2009_2
train_2009['distance_5']  = results_2009_3

train_09 = train[train['year'] == 2009]
train_09.reset_index(inplace = True)
train_9 = pd.concat([train_09, train_2009], axis = 1)

CPU times: user 4.73 s, sys: 4.06 ms, total: 4.74 s
Wall time: 4.74 s


In [45]:
%%time

# Using 2011 Train data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2011 = train[train['year'] == 2011.0]
features = ['latitude', 'longitude']

distance_2011 = distance_2011[features]
distance_2011 = distance_2011.apply(tuple, axis = 1)
train['wnvpresent'] = train['wnvpresent'].astype(float)

westnile_2011 = (train[train['wnvpresent'] == 1]) 
westnile_2011 = westnile_2011[westnile_2011['year'] == 2011.0]
westnile_2011 = westnile_2011[features]
westnile_2011 = westnile_2011.apply(tuple, axis = 1)
westnile_2011.head()


results_2011   = []
results_2011_2 = []
results_2011_3 = []

for i in distance_2011:
    dists_2011 = []

    for j in westnile_2011:

        dist_2011 = (geodesic(i, j).miles)
        if dist_2011 > 0:
            dists_2011.append(dist_2011)
   
    results_2011.append(min(dists_2011))
    results_2011_2.append(sum(dists_2011[0:3])/3)
    results_2011_3.append(sum(dists_2011[0:5])/5)
    
# Create dataframe    
train_2011=pd.DataFrame(results_2011, columns = ['distance'])
train_2011['distance_3']  = results_2011_2
train_2011['distance_5']  = results_2011_3                                              
train_11 = train[train['year'] == 2011]
train_11.reset_index(inplace = True)
train_1 = pd.concat([train_11, train_2011], axis = 1)

CPU times: user 10.3 s, sys: 8.84 ms, total: 10.3 s
Wall time: 10.3 s


In [46]:
%%time

# Using 2013 Train data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2013 = train[train['year'] == 2013.0]
features = ['latitude', 'longitude']

distance_2013 = distance_2013[features]
distance_2013 = distance_2013.apply(tuple, axis = 1)
train['wnvpresent'] = train['wnvpresent'].astype(float)

westnile_2013 = (train[train['wnvpresent'] == 1]) 
westnile_2013 = westnile_2013[westnile_2013['year'] == 2013.0]
westnile_2013 = westnile_2013[features]
westnile_2013 = westnile_2013.apply(tuple, axis = 1)
westnile_2013.head()


results_2013   = []
results_2013_2 = []
results_2013_3 = []

for i in distance_2013:
    dists_2013 = []

    for j in westnile_2013:

        dist_2013 = (geodesic(i, j).miles)
        if dist_2013 > 0:
            dists_2013.append(dist_2013)
   
    results_2013.append(min(dists_2013))
    results_2013_2.append(sum(dists_2013[0:3])/3)
    results_2013_3.append(sum(dists_2013[0:5])/5)
    
# Create dataframe    
train_2013=pd.DataFrame(results_2013, columns = ['distance'])
train_2013['distance_3']  = results_2013_2
train_2013['distance_5']  = results_2013_3                                              
train_13 = train[train['year'] == 2013]
train_13.reset_index(inplace = True)
train_3 = pd.concat([train_13, train_2013], axis = 1)

CPU times: user 56.8 s, sys: 24.2 ms, total: 56.8 s
Wall time: 56.8 s


In [47]:
distance_2013.shape

(2012,)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print())? (<ipython-input-48-f7f5dc6bf9bd>, line 1)

In [49]:
# Take each year of train distance data just calculated and add to new df
list = [train_7, train_9, train_1, train_3]
train_new = pd.concat(list, axis = 0, )
train_new


,index,date,address,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,...,avgspeed_y,species_culex_pipiens,species_culex_pipiens/restuans,species_culex_restuans,species_culex_salinarius,species_culex_tarsalis,species_culex_territans,distance,distance_3,distance_5
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,7.4,0,1,0,0,0,0,1.050839,1.117797,1.176855
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,7.4,0,0,1,0,0,0,1.050839,1.117797,1.176855
2,2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,7,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,7.4,0,0,1,0,0,0,1.021493,1.021493,1.050681
3,3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,7.4,0,1,0,0,0,0,0.822410,0.822410,1.218078
4,4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,7.4,0,0,1,0,0,0,0.822410,0.822410,1.218078
5,5,2007-05-29,"1500 West Webster Avenue, Chicago, IL 60614, USA",15,W WEBSTER AVE,45,"1500 W WEBSTER AVE, Chicago, IL",41.921600,-87.666455,8,...,7.4,0,0,1,0,0,0,1.706440,1.706440,1.807555
6,6,2007-05-29,"2500 West Grand Avenue, Chicago, IL 60654, USA",25,W GRAND AVE,46,"2500 W GRAND AVE, Chicago, IL",41.891118,-87.654491,8,...,7.4,0,0,1,0,0,0,0.358443,1.191190,1.388980
7,7,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",11,W ROOSEVELT,48,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,...,7.4,0,1,0,0,0,0,0.432500,0.853708,1.400547
8,8,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",11,W ROOSEVELT,48,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,...,7.4,0,0,1,0,0,0,0.432500,0.853708,1.400547
9,9,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",11,W CHICAGO,49,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,...,7.4,0,0,1,0,0,0,1.765359,1.765359,1.790267


In [50]:
train.shape

(8475, 60)

### Repeating the Geopy process with the Test Data

In [51]:
# test['distance'] = 0

In [52]:
%%time

# Using 2008 Test data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2008 = test[test['year'] == 2008.0]
features = ['latitude', 'longitude']

distance_2008 = distance_2008[features]
distance_2008 = distance_2008.apply(tuple, axis = 1)

westnile_2007 = (train[train['wnvpresent'] == 1]) 
westnile_2007 = westnile_2007[westnile_2007['year'] == 2007.0]
westnile_2007 = westnile_2007[features]
westnile_2007 = westnile_2007.apply(tuple, axis = 1)


results_2008   = []
results_2008_2 = []
results_2008_3 = []

for i in distance_2008:
    dists_2008 = []

    for j in westnile_2007:

        dist_2008 = (geodesic(i, j).miles)
        if dist_2008 > 0:
            dists_2008.append(dist_2008)
    
    
    results_2008.append(min(dists_2008))
    results_2008_2.append(sum(dists_2008[0:3])/3)
    results_2008_3.append(sum(dists_2008[0:5])/5)
    
# Create dataframe    
test_2008=pd.DataFrame(results_2008, columns = ['distance'])
test_2008['distance_3']  = results_2008_2
test_2008['distance_5']  = results_2008_3                                              
test_08 = test[test['year'] == 2008]
test_08.reset_index(inplace = True)
test_8 = pd.concat([test_08, test_2008], axis = 1)

CPU times: user 11min 53s, sys: 553 ms, total: 11min 54s
Wall time: 15min 25s


In [53]:
%%time

# Using 2010 Test data, find the distance between mosquitos that tested positive for WNV vs. those that did not


distance_2010 = test[test['year'] == 2010.0]
features = ['latitude', 'longitude']

distance_2010 = distance_2010[features]
distance_2010 = distance_2010.apply(tuple, axis = 1)

westnile_2009 = (train[train['wnvpresent'] == 1]) 
westnile_2009 = westnile_2009[westnile_2009['year'] == 2007.0]
westnile_2009 = westnile_2009[features]
westnile_2009 = westnile_2009.apply(tuple, axis = 1)


results_2010   = []
results_2010_2 = []
results_2010_3 = []

for i in distance_2010:
    dists_2010= []

    for j in westnile_2009:

        dist_2010 = (geodesic(i, j).miles)
        if dist_2010 > 0:
            dists_2010.append(dist_2010)
   
    results_2010.append(min(dists_2010))
    results_2010_2.append(sum(dists_2010[0:3])/3)
    results_2010_3.append(sum(dists_2010[0:5])/5)
    
# Create dataframe    
test_2010=pd.DataFrame(results_2010, columns = ['distance'])
test_2010['distance_3']  = results_2010_2
test_2010['distance_5']  = results_2010_3                                              
test_10 = test[test['year'] == 2010]
test_10.reset_index(inplace = True)
test_0 = pd.concat([test_10, test_2010], axis = 1)

CPU times: user 14min 9s, sys: 588 ms, total: 14min 10s
Wall time: 14min 10s


In [55]:
%%time

# Using 2012 Test data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2012 = test[test['year'] == 2012.0]
features = ['latitude', 'longitude']

distance_2012 = distance_2012[features]
distance_2012 = distance_2012.apply(tuple, axis = 1)


westnile_2011 = (train[train['wnvpresent'] == 1]) 
westnile_2011 = westnile_2011[westnile_2011['year'] == 2011.0]
westnile_2011 = westnile_2011[features]
westnile_2011 = westnile_2011.apply(tuple, axis = 1)

results_2012   = []
results_2012_2 = []
results_2012_3 = []

for i in distance_2012:
    dists_2012 = []

    for j in westnile_2011:

        dist_2012 = (geodesic(i, j).miles)
        if dist_2012 > 0:
            dists_2012.append(dist_2012)
   
    results_2012.append(min(dists_2012))
    results_2012_2.append(sum(dists_2012[0:3])/3)
    results_2012_3.append(sum(dists_2012[0:5])/5)
    
# Create dataframe    
test_2012=pd.DataFrame(results_2012, columns = ['distance'])
test_2012['distance_3']  = results_2012_2
test_2012['distance_5']  = results_2012_3                                              
test_12 = test[test['year'] == 2012]
test_12.reset_index(inplace = True)
test_2 = pd.concat([test_12, test_2012], axis = 1)

CPU times: user 2min 36s, sys: 80 ms, total: 2min 36s
Wall time: 2min 36s


In [54]:
%%time

# Using 2014 Test data, find the distance between mosquitos that tested positive for WNV vs. those that did not

distance_2014 = test[test['year'] == 2014.0]
features = ['latitude', 'longitude']

distance_2014 = distance_2014[features]
distance_2014 = distance_2014.apply(tuple, axis = 1)

westnile_2013 = (train[train['wnvpresent'] == 1]) 
westnile_2013 = westnile_2013[westnile_2013['year'] == 2013.0]
westnile_2013 = westnile_2013[features]
westnile_2013 = westnile_2013.apply(tuple, axis = 1)

results_2014   = []
results_2014_2 = []
results_2014_3 = []
for i in distance_2014:
    dists_2014 = []

    for j in westnile_2013:

        dist_2014 = (geodesic(i, j).miles)
        if dist_2014 > 0:
            dists_2014.append(dist_2014)
   
    results_2014.append(min(dists_2014))
    results_2014_2.append(sum(dists_2014[0:3])/3)
    results_2014_3.append(sum(dists_2014[0:5])/5)
    
# Create dataframe    
test_2014=pd.DataFrame(results_2014, columns = ['distance'])
test_2014['distance_3']  = results_2014_2
test_2014['distance_5']  = results_2014_3                                              
test_14 = test[test['year'] == 2014]
test_14.reset_index(inplace = True)
test_4 = pd.concat([test_14, test_2014], axis = 1)

CPU times: user 10min 24s, sys: 441 ms, total: 10min 24s
Wall time: 13min 30s


In [56]:
# Take each year of test distance data just calculated and add to new df
list = [test_8, test_0, test_2, test_4]
test_new = pd.concat(list, axis = 0)
test_new.head()

,index,id,date,address,block,street,trap,addressnumberandstreet,latitude,longitude,...,species_culex_pipiens,species_culex_pipiens/restuans,species_culex_restuans,species_culex_salinarius,species_culex_tarsalis,species_culex_territans,species_unspecified_culex,distance,distance_3,distance_5
0,0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,0,1,0,0,0,0,0,1.050839,10.042778,8.721685
1,1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,0,0,1,0,0,0,0,1.050839,10.042778,8.721685
2,2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,1,0,0,0,0,0,0,1.050839,10.042778,8.721685
3,3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,0,0,0,1,0,0,0,1.050839,10.042778,8.721685
4,4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,0,0,0,0,0,1,0,1.050839,10.042778,8.721685


In [58]:
test_new

,index,id,date,address,block,street,trap,addressnumberandstreet,latitude,longitude,...,species_culex_pipiens,species_culex_pipiens/restuans,species_culex_restuans,species_culex_salinarius,species_culex_tarsalis,species_culex_territans,species_unspecified_culex,distance,distance_3,distance_5
0,0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,1,0,0,0,0,0,1.050839,10.042778,8.721685
1,1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,1,0,0,0,0,1.050839,10.042778,8.721685
2,2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,1,0,0,0,0,0,0,1.050839,10.042778,8.721685
3,3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,0,1,0,0,0,1.050839,10.042778,8.721685
4,4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,0,0,0,1,0,1.050839,10.042778,8.721685
5,5,6,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,0,0,1,0,0,1.050839,10.042778,8.721685
6,6,7,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,0,0,0,0,1,1.050839,10.042778,8.721685
7,7,8,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,0,0,0,0,0,0,0,1.050839,10.042778,8.721685
8,8,9,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,7,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,...,0,1,0,0,0,0,0,1.021493,9.548328,9.526683
9,9,10,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,7,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,...,0,0,1,0,0,0,0,1.021493,9.548328,9.526683


In [ ]:
test_new.shape

In [ ]:
train_new.shape

In [ ]:
# Save cleaned files
spray.to_csv('../project_work/data/sprayclean.csv', index=False)
weather.to_csv('../project_work/data/weatherclean.csv', index=False)
train_new.to_csv('../project_work/data/trainclean.csv', index=False)
test_new.to_csv('../project_work/data/testclean.csv', index=False)